# Loading a dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

raw_inputs = load_dataset('glue', 'mrpc')
raw_inputs

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
raw_inputs_train = raw_inputs['train']
raw_inputs_train[15]

{'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
 'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .',
 'label': 0,
 'idx': 16}

In [ ]:
def tokenize_function(data):
  return tokenizer(data['sentence1'], data['sentence2'], truncation = True)

In [ ]:
tokenized_dataset = raw_inputs.map(tokenize_function, batched = True)
print(tokenized_dataset)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})


In [ ]:
sample = tokenized_dataset['train'][:8]
sample = {k: v for k, v  in sample.items() if k not in ['idx', 'sentence1', 'sentence2']}
[len(x) for x in sample['input_ids']]

[50, 59, 47, 67, 59, 50, 62, 32]

Note that the sequences length are not all equal. To make the length equal we have to do padding. This is where we use data collator which will pad each sequence to max_length of a sequence in a batch else it would have padded the sequence to max_length of the model

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer)

# Training

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 1.9 MB/s eta 0:00:00


In [ ]:
pip install accelerate>=0.20.1 -U

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments('test_trainer')

In [ ]:
#Load the model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation'],
    data_collator = data_collator,
    tokenizer = tokenizer
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.213100
1000,0.123100


TrainOutput(global_step=1377, training_loss=0.13416728128448463, metrics={'train_runtime': 8983.137, 'train_samples_per_second': 1.225, 'train_steps_per_second': 0.153, 'total_flos': 406683358271520.0, 'train_loss': 0.13416728128448463, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(tokenized_dataset['validation'])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [ ]:
predictions

PredictionOutput(predictions=array([[-3.5683422 ,  4.150214  ],
       [ 2.851807  , -4.0160356 ],
       [ 2.074573  , -3.379192  ],
       [-3.5313087 ,  4.161585  ],
       [ 2.6572866 , -3.7839472 ],
       [-3.554834  ,  4.1609836 ],
       [-3.3307338 ,  3.9971168 ],
       [-3.5584419 ,  4.172943  ],
       [-3.5170481 ,  4.1475677 ],
       [-3.5825288 ,  4.154045  ],
       [-3.5802784 ,  4.139841  ],
       [ 2.283385  , -3.352879  ],
       [ 2.8030677 , -3.8917937 ],
       [-2.117405  ,  2.491846  ],
       [-3.5818448 ,  4.141988  ],
       [-0.13541499, -0.16523193],
       [-3.573346  ,  4.1149244 ],
       [ 1.842675  , -2.968649  ],
       [-3.556928  ,  4.143878  ],
       [ 2.8996062 , -3.8810153 ],
       [ 2.9119985 , -3.950842  ],
       [-2.948874  ,  3.5755363 ],
       [ 2.9025478 , -3.6429043 ],
       [-3.5586839 ,  4.152455  ],
       [-3.4840345 ,  4.161509  ],
       [-1.451178  ,  1.5113877 ],
       [ 1.228164  , -1.4218638 ],
       [-3.5743449 ,  4.12

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis = -1)

In [ ]:
preds

array([1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,

In [ ]:

import numpy as np

def compute_metrics(eval_preds):
  metric = evaluate.load('glue', 'mrpc')
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis = -1)
  return metric.compute(predictions = predictions, references = labels)

In [ ]:
metric = evaluate.load('glue', 'mrpc')
metric.compute(predictions = preds, references = predictions.label_ids)